# Projeto: Previsão de Preços de Imóveis em Ames

### Importação das Bibliotecas Necessárias

In [ ]:
# Importação das bibliotecas essenciais
import pickle
import pandas as pd
from pathlib import Path

# Importação das bibliotecas de pré-processamento e modelagem
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Importação dos modelos de regressão
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

# Importação das métricas de avaliação
from sklearn.metrics import mean_squared_error, r2_score

# Importação para visualização dos resultados
import matplotlib.pyplot as plt
import seaborn as sns

### Carregamento dos Dados Processados

In [ ]:
# Definição do caminho para o arquivo processado
DATA_DIR = Path.cwd().parent / 'data'
processed_file_path = DATA_DIR / 'processed' / 'ames_clean.pkl'

# Carregamento dos dados processados
with open(processed_file_path, 'rb') as file:
    data = pickle.load(file)

### Divisão dos Dados em Conjuntos de Treinamento e Teste

In [ ]:
# Definição da variável alvo
y = data['SalePrice']

# Definição das variáveis preditoras
X = data.drop('SalePrice', axis=1)

# Divisão dos dados em treinamento e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### Identificação das Colunas Categóricas e Numéricas

In [ ]:
# Identificação das colunas categóricas e numéricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Colunas Categóricas: {categorical_cols}")
print(f"Colunas Numéricas: {numerical_cols}")

### Conversão das Colunas Categóricas para Strings

In [ ]:
# Converter todas as colunas categóricas para string para evitar erros no encoder
X_train[categorical_cols] = X_train[categorical_cols].astype(str)
X_test[categorical_cols] = X_test[categorical_cols].astype(str)

# Verificação para garantir que a conversão foi bem-sucedida
print(X_train[categorical_cols].dtypes)
print(X_test[categorical_cols].dtypes)

In [ ]:
def percentage_error(y_true, y_pred):
    """Calcula o Erro Percentual Absoluto Médio (PEA).

    Args:
        y_true (array-like): Valores reais.
        y_pred (array-like): Valores previstos.

    Returns:
        float: PEA em porcentagem.
    """
    y_true, y_pred = pd.Series(y_true), pd.Series(y_pred)
    # Evitar divisão por zero substituindo zeros por um pequeno valor
    y_true = y_true.replace(0, 1e-10)
    return (abs((y_true - y_pred) / y_true).mean()) * 100

### Definição das Pipelines de Pré-processamento

In [ ]:
# Pipeline para variáveis numéricas
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para variáveis categóricas
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinação das pipelines usando ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

### Definição dos Modelos

In [ ]:
# Definição dos modelos a serem comparados com seus hiperparâmetros
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Ridge Regression': {
        'model': Ridge(),
        'params': {
            'regressor__alpha': [0.1, 1.0, 10.0, 100.0]
        }
    },
    'Lasso Regression': {
        'model': Lasso(),
        'params': {
            'regressor__alpha': [0.01, 0.1, 1.0, 10.0]
        }
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {
            'regressor__n_estimators': [100, 200],
            'regressor__max_depth': [None, 10, 20],
            'regressor__min_samples_split': [2, 5],
            'regressor__min_samples_leaf': [1, 2]
        }
    }
}

### Criação das Pipelines de Modelos

In [ ]:
# Criação das pipelines e configuração do GridSearchCV
pipelines = {}
for name, config in models.items():
    # Pipeline para cada modelo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', config['model'])
    ])
    
    # GridSearchCV para otimização de hiperparâmetros
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=config['params'],
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )
    
    pipelines[name] = grid_search

### Treinamento dos Modelos

In [ ]:
# Treinamento de cada modelo com GridSearchCV
for name, grid_search in pipelines.items():
    try:
        print(f"Iniciando treinamento do modelo: {name}")
        grid_search.fit(X_train, y_train)
        print(f"{name} treinado com sucesso.\n")
    except Exception as e:
        print(f"Erro ao treinar {name}: {e}\n")

### Avaliação e Comparação dos Modelos

In [ ]:
# Avaliação dos modelos no conjunto de teste
results = []

for name, grid_search in pipelines.items():
    try:
        # Previsões
        y_pred = grid_search.predict(X_test)
        
        # Cálculo das métricas
        r2 = r2_score(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        percent_error = percentage_error(y_test, y_pred)  # Erro percentual em porcentagem
        
        # Coleta dos melhores hiperparâmetros
        best_params = grid_search.best_params_
        
        # Armazenamento dos resultados
        results.append({
            'Modelo': name,
            'RMSE': f"{rmse:.2f}",
            'Erro Percentual (%)': f"{percent_error:.2f}%",
            'Melhores Hiperparâmetros': best_params
        })
        
        print(f"{name} avaliado com sucesso.")
    except Exception as e:
        print(f"Erro ao avaliar {name}: {e}")

# Conversão da lista de resultados para DataFrame
results_df = pd.DataFrame(results)

# Exibição dos resultados ordenados por Erro Percentual
print("\nResultados dos Modelos:")
print(results_df[['Modelo', 'RMSE', 'Erro Percentual (%)', 'Melhores Hiperparâmetros']].sort_values(by='Erro Percentual (%)'))

### Visualização dos Resultados

In [ ]:
# Configuração do estilo dos gráficos
sns.set_theme(style="whitegrid")

# Gráfico de barras para RMSE
plt.figure(figsize=(10,6))
sns.barplot(x='RMSE', y='Modelo', data=results_df.sort_values('RMSE', ascending=True), palette='viridis')
plt.title('Desempenho dos Modelos - RMSE')
plt.xlabel('RMSE')
plt.ylabel('Modelo')
plt.show()

# Gráfico de barras para Erro Percentual
plt.figure(figsize=(10,6))
sns.barplot(x='Erro Percentual (%)', y='Modelo', data=results_df.sort_values('Erro Percentual (%)', ascending=True), palette='magma')
plt.title('Desempenho dos Modelos - Erro Percentual (%)')
plt.xlabel('Erro Percentual (%)')
plt.ylabel('Modelo')
plt.show()

### Identificação das 10 Features Mais Importantes

In [ ]:
# Inicialização de um dicionário para armazenar as importâncias das features
feature_importances = pd.Series(dtype=float)

for name, grid_search in pipelines.items():
    try:
        model = grid_search.best_estimator_.named_steps['regressor']
        
        # Obter nomes das features após OneHotEncoder
        if hasattr(grid_search.best_estimator_.named_steps['preprocessor'], 'transformers_'):
            ohe = grid_search.best_estimator_.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
            ohe_features = ohe.get_feature_names_out(categorical_cols)
            all_features = list(ohe_features) + numerical_cols
        else:
            all_features = numerical_cols
        
        if hasattr(model, 'feature_importances_'):
            # Para modelos como Random Forest
            importances = pd.Series(model.feature_importances_, index=all_features)
        elif hasattr(model, 'coef_'):
            # Para modelos lineares
            importances = pd.Series(abs(model.coef_), index=all_features)
        else:
            print(f"{name} não possui atributo de importância das features.")
            continue
        
        # Normalização das importâncias
        importances = importances / importances.sum()
        
        # Acumulação das importâncias
        feature_importances = feature_importances.add(importances, fill_value=0)
    except Exception as e:
        print(f"Erro ao coletar importâncias das features para {name}: {e}")

# Média das importâncias
feature_importances = feature_importances / len(pipelines)

# Seleção das 10 features mais importantes
top_10_features = feature_importances.sort_values(ascending=False).head(10)

print("\nTop 10 Features Mais Importantes:")
print(top_10_features)

### Exibição das 10 Features Mais Importantes

In [ ]:
# Visualização das 10 features mais importantes
plt.figure(figsize=(10,6))
sns.barplot(x=top_10_features.values, y=top_10_features.index, palette='viridis')
plt.title('Top 10 Features Mais Importantes')
plt.xlabel('Importância Média')
plt.ylabel('Feature')
plt.show()

### Comparação Final entre os Modelos

In [ ]:
# Identificação do modelo com menor Erro Percentual
best_model = results_df.loc[results_df['Erro Percentual (%)'].astype(float).idxmin()]

print("\nModelo com menor erro percentual (Erro Percentual):")
print(f"Modelo: {best_model['Modelo']}")
print(f"RMSE: {best_model['RMSE']}")
print(f"Erro Percentual: {best_model['Erro Percentual (%)']}")
print(f"Melhores Hiperparâmetros: {best_model['Melhores Hiperparâmetros']}")